- 联合分布的对数似然为 $\log p(x_1, x_2, P)$。其 ELBO 定义为：
    - $\mathcal{L}(q) = \mathbb{E}_{q(W, z_1, z_2)}[\log p(x_1, x_2, P, W, z_1, z_2)] - \mathbb{E}_{q(W, z_1, z_2)}[\log q(W, z_1, z_2)]$
    - $q(W, z_1, z_2) = q(W) q(z_1) q(z_2)$
    - $\mathcal{L}(q) = \mathbb{E}_{q(W)q(z_1)q(z_2)} [ \log p(x_1|z_1) + \log p(x_2|z_2) + \log p(P|z_1, z_2, W) ] - D_{KL}(q(W, z_1, z_2) || p(W, z_1, z_2))$
        - $D_{KL}(q(W, z_1, z_2) || p(W, z_1, z_2)) = D_{KL}(q(W) || p(W)) + D_{KL}(q(z_1) || p(z_1)) + D_{KL}(q(z_2) || p(z_2))$
    - $z$：分两部分一部分用来生成（重构数据$x$），一部分用来生成 preference $P$

$$
\begin{split}
\mathcal{L} &= \underbrace{\mathbb{E}_{q(z_{1,x})}[\log p(x_1|z_{1,x})] + \mathbb{E}_{q(z_{2,x})}[\log p(x_2|z_{2,x})]}_{\text{reconstruction}} \\
&+ \underbrace{\mathbb{E}_{q(W)q(z_{1,p})q(z_{2,p})}[\log p(P|z_{1,p}, z_{2,p}, W)]}_{\text{Preference}} \\
&- \underbrace{D_{KL}(q(W)||p(W)) - D_{KL}(q(z_1)||p(z_1)) - D_{KL}(q(z_2)||p(z_2))}_{\text{KL reg}}
\end{split}
$$

